<a href="https://colab.research.google.com/github/SarsijNayan/earthengine-community/blob/master/IMDLIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Connect Google drive with Google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
pip install imdlib

In [7]:
# Download the IMD raw data into the google drive
import imdlib as imd
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

path = "/content/drive/MyDrive/Colab Notebooks"

start_yr = 2019
end_yr = 2024
variable = 'rain' # other options are ('tmin'/ 'tmax')

imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=path)
data = imd.open_data(variable, start_yr, end_yr,'yearwise', path)
ds = data.get_xarray()
print(ds)



Downloading: rain for year 2019
Downloading: rain for year 2020
Downloading: rain for year 2021
Downloading: rain for year 2022
Downloading: rain for year 2023
Downloading: rain for year 2024
Download Successful !!!
<xarray.Dataset> Size: 305MB
Dimensions:  (time: 2192, lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 1kB 6.5 6.75 7.0 7.25 7.5 ... 37.75 38.0 38.25 38.5
  * lon      (lon) float64 1kB 66.5 66.75 67.0 67.25 ... 99.25 99.5 99.75 100.0
  * time     (time) datetime64[ns] 18kB 2019-01-01 2019-01-02 ... 2024-12-31
Data variables:
    rain     (time, lat, lon) float64 305MB -999.0 -999.0 ... -999.0 -999.0
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2025-03-10 04:52:44.342251 Python
    references:   
    comment:      
    crs:          epsg:4326


In [8]:
# Provide the alttitude & Longitude of a point for which the data is required
#  And save the data into CSV file

lat = 25.90 #lattitude of point
lon = 85.10 #longitude of point
data.to_csv('data.csv', lat, lon, path)

In [9]:
# Save CSV files for multiple points

# Provide lat and long in a list
latLong = [
    [25.90, 85.10],  # Patna (Central Bihar)
    [25.50, 86.98],  # Bhagalpur
    [26.12, 84.20],  # Gopalganj
]

for points in latLong:
  lat = points[0]
  lon = points[1]

  data.to_csv('test.csv', lat, lon, path)
  print ("data save for ",points)

data save for  [25.9, 85.1]
data save for  [25.5, 86.98]
data save for  [26.12, 84.2]


In [15]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

# Provide the new GeoJSON file link
geojson_url = "https://drive.google.com/uc?id=" + "1KjYAcFomJwLQf1Q-JRNJJGpwTzKiecmd"

# Download the GeoJSON file
geojson_file = 'Test_geojson.geojson'
response = requests.get(geojson_url)
with open(geojson_file, 'wb') as file:
    file.write(response.content)

# Data points CSV link
url = "https://drive.google.com/uc?id=111XvmUzvTlhY2lbFMseGVhZQh4pisFXQ"  # Replace with your data points file link
points_df = pd.read_csv(url)

# Creating geometry column for the points dataframe
geometry = [Point(xy) for xy in zip(points_df['Long'], points_df['Lat'])]

# Creating GeoDataFrame for points
gdf_points = gpd.GeoDataFrame(points_df, geometry=geometry)

# Set CRS (EPSG:4326 is WGS84 Lat/Long)
gdf_points.set_crs(epsg=4326, inplace=True)

# Load the GeoJSON polygon file as GeoDataFrame
gdf_polygon = gpd.read_file(geojson_file)

# Ensure both GeoDataFrames have the same CRS
if gdf_points.crs != gdf_polygon.crs:
    gdf_points = gdf_points.to_crs(gdf_polygon.crs)

# Find points lying within the polygon
gdf_list = []
for row in range(len(gdf_polygon)):
    points_in_polygon = gdf_points[gdf_points.within(gdf_polygon.iloc[row].geometry)]
    gdf_list.append(points_in_polygon)

# Combine all points within polygons into a single GeoDataFrame
final_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# Save the filtered data to a CSV file
final_df = final_gdf[["Name", "Lat", "Long"]]
final_df.to_csv("Master_file.csv", index=False)

# Save gridded data for each point
for index, row in final_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    # Assuming `data` contains the gridded data to be saved
    data.to_csv(f"test_{lat}_{lon}.csv", lat, lon, path)
    print("Data saved for point:", lat, lon)


Data saved for point: 24.5 84.5
Data saved for point: 24.5 84.75
Data saved for point: 24.5 85.0
Data saved for point: 24.5 85.25
Data saved for point: 24.75 83.5
Data saved for point: 24.75 83.75
Data saved for point: 24.75 84.0
Data saved for point: 24.75 84.25
Data saved for point: 24.75 84.5
Data saved for point: 24.75 84.75
Data saved for point: 24.75 85.0
Data saved for point: 24.75 85.25
Data saved for point: 24.75 85.5
Data saved for point: 24.75 86.25
Data saved for point: 24.75 86.5
Data saved for point: 24.75 86.75
Data saved for point: 24.75 87.0
Data saved for point: 25.0 83.5
Data saved for point: 25.0 83.75
Data saved for point: 25.0 84.0
Data saved for point: 25.0 84.25
Data saved for point: 25.0 84.5
Data saved for point: 25.0 84.75
Data saved for point: 25.0 85.0
Data saved for point: 25.0 85.25
Data saved for point: 25.0 85.5
Data saved for point: 25.0 85.75
Data saved for point: 25.0 86.0
Data saved for point: 25.0 86.25
Data saved for point: 25.0 86.5
Data saved fo